# Visualising Weather prediction on Omniverse - FourCastNet

This notebook will give you a start in obtaining inference results from FourCastNet and visualising the results on our Omniverse Digital Twin.

#### Contents of the Notebook

- [Building an extension to visualise weather prediction using FourCastNet](#Building-an-extension-to-visualise-weather-prediction-using-FourCastNet)
    - [Obtaining results from FourCastNet](#Obtaining-results-from-FourCastNet)
    - [Visualising results in Omniverse](#Visualising-results-in-Omniverse)
    - [Mini challenge - Visualising the Total column water vapour channel](#Mini-challenge---Visualising-Total-Column-Water-Vapour)
        
#### Learning Outcomes
- How to get started in Omniverse
- Visualising our data using custom-built extensions.

## Building an extension to visualise weather prediction using FourCastNet

Let us follow a step-by-step approach to building an extension on Omniverse and use the results obtained from FourCastNet to visualise a Digital Twin of Earth.

These steps assume you have followed the previous notebook (Visualising Navier-Stoke weather prediction on Omniverse) to complete the first set of visualisation. Before we proceed forward to modify our plugin, let us first generate results from a Trained FourCastNet Model. The Trained FourCastNet model is available to us as an open-source repository on GitHub - [FourCastNet](https://github.com/NVlabs/FourCastNet)

### Obtaining results from FourCastNet 

Let us now run the inference for the FourCastNet model to obtain the outputs, which we can further process and visualise in our system.

#### Optional - Visualising real-time results - Downloading dataset 

A sample of data for one day is already downloaded and present in the container, but if the container has access to the Internet, you can download the latest data available to see predictions on the day you are running the material. 

- **Steps to download the latest data available to us** 
    - Create a new account on the [Copernicus Climate Change Service Data Store](https://cds.climate.copernicus.eu/user/register?destination=%2F%23!%2Fhome) and log in to your account
    - Head over to [ERA5 Reanalysis hourly pressure data](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels?tab=form) page and set the year and month column to find out what is the latest data available, this dataset is updated with a latency of 5 days. Kindly make a note of the date available to us. For this example, we will use **March 27 2023**. 
    - Modify the files accordingly to use the latest date available and set the appropriate location to download and store the data [get_data_pl_short_length.py](../../source_code/FCN/copernicus/get_data_pl_short_length.py) & [get_data_sfc_short_length.py](../../source_code/FCN/copernicus/get_data_sfc_short_length.py)
    - Obtain your API key from [here](https://cds.climate.copernicus.eu/api-how-to)
    - Open a new Terminal and create a new file ( `touch $HOME/.cdsapirc` ) and add your API key ( `nano $HOME/.cdsapirc` ), and save and close. 
    - Download the data by running the cell below
    - Verify if all the locations are correct in our data_preprocessing script present [here](../../source_code/FCN/data_process/parallel_copy_small_set.py) and run the script to make them compatible with the FourCastNet format.

In [ ]:
# Optional 
# Download PL and SFC for building our inference data 
!python3 ../../source_code/FCN/copernicus/get_data_pl_short_length.py && python3 ../../source_code/FCN/copernicus/get_data_sfc_short_length.py

In [ ]:
# Optional
# Pre-processing the downloaded data to make it compatible with FourCastNet 
!python3 ../../source_code/FCN/data_process/parallel_copy_small_set.py

#### Running inference on the dataset

Let us now run an inference on the dataset using the FourCastNet Model. 

In [ ]:
!cd ../../source_code/FCN && python3 inference/inference.py --vis --config=afno_backbone --run_num=0 \
        --weights "/workspace/python/source_code/FCN/FCN_weights_v0/backbone.ckpt" \
        --override_dir "/workspace/python/source_code/FCN/output"

Now that we have run our inference, we can see the first three days have minimal error, because they are the prediction for Day 0 at different time steps whereas, the remaining predictions are compared to a dummy value, Let us now convert these predictions to images and have them stored for visualisation.

In [ ]:
# Converting the outputs from h5py file to images and storing them in the assets folder
import h5py
import os
import numpy as np
import multiprocessing
from tqdm import tqdm
import matplotlib.pyplot as plt

variable=['u10','v10','t2m','sp','msl','t_850','u_1000','v_1000','z_1000','u_850','v_850',
            'z_850','u_500','v_500','z_500','t_500','z_50','r_500','r_850','tcwv']

filename = "../../source_code/FCN/output/autoregressive_predictions_z500_vis.h5"
hf = h5py.File(filename, 'r')
data = hf.get('predicted')[0]

##### Setting j=0 to visualise u10 (Eastward component of wind at sea level)
j=0
#####

for i in tqdm(range(data.shape[0])):
    # Get the graph 
    plt.imshow(data[i][j],cmap='gray')
    plt.axis('off')
    # Save as image
    store_loc = "/../../source_code/extension/assets/FCN_output/"+variable[j]+'/'
    if not os.path.exists(os.getcwd()+store_loc):
        os.makedirs(os.getcwd()+store_loc)
    plt.savefig(os.getcwd()+store_loc+str(i)+'.jpg',bbox_inches='tight', pad_inches=0)
plt.close()

### Visualising results in Omniverse


#### 1. Modify the extension 

We can either create a new extension or modify the existing extension for FourCastNet. In this case, we will be modifying the existing extension. If needed, you can go ahead to create a new extension. 

Open the [`extension.py`](../../source_code/extension/omniverse-project-navier-stokes2/exts/omniverse.extension.navierstokes/omniverse/extension/navierstokes/extension.py) file for modifications. 

Let us now change the directory and TimeCode to reflect the results of FourCastNet.


```python
import omni.ext
import omni.ui as ui
from pxr import Usd, UsdShade, Sdf
import numpy as np
import os 

# Any class derived from `omni.ext.IExt` in top level module (defined in `python.modules` of `extension.toml`) will be
# instantiated when extension gets enabled and `on_startup(ext_id)` will be called. Later when extension gets disabled
# on_shutdown() is called.
class OmniverseExtensionNavierstokesExtension(omni.ext.IExt):
    # ext_id is current extension id. It can be used with extension manager to query additional information, like where
    # this extension is located on filesystem.
    def on_startup(self, ext_id):
        print("[omniverse.extension.navierstokes] omniverse extension navierstokes startup")
        
        # Get shader input for opacity map
        stage = omni.usd.get_context().get_stage()
        shader = UsdShade.Shader.Get(stage,"/World/Looks/Clouds/Shader")
        texture = shader.GetInput("opacity_texture")

        # Get and store image paths 
        files=[]
        for i in range(30):
            files.append(os.getcwd()+'/../extension/assets/FCN_output/u10/'+str(i)+'.jpg')
        
        # Assign images every 5 timesteps
        texture.GetAttr().Clear()
        for i in range(30):
            texture.Set(Sdf.AssetPath(files[i]),Usd.TimeCode(5*i))
                        
        # Trim Timeline
        stage.SetStartTimeCode(0)
        stage.SetEndTimeCode(150)
        
        
    def on_shutdown(self):
        print("[omniverse.extension.navierstokes] omniverse extension navierstokes shutdown")
```

#### 2. Setting "Opacity Map Color Space" to `auto`

We can now set the `Opacity Map Color Space` to `auto` for the application to auto-detect the color space of our textures and apply them on top of the Digital Twin. 

<center><img src="images/ext10.png" alt="Drawing" style="width:1000px" /></center>

#### 3. Playing the animation

You can play the animation by clicking on the `Play` button present on the left side of the taskbar. 

The first time we run it, it takes a bit of time to load all the textures, after which the animations fell smoother once loaded.

In [1]:
from IPython.display import Video
Video("images/fcn.mp4",width=1200, height=600,html_attributes='controls loop autoplay')

## Mini challenge - Visualising Total Column Water Vapour

Similar to the approach above, the Mini challenge is to visualize the Total column water vapour variable using Omniverse. You will be provided with snippets that you can fill in the TODO blanks to solve the challenge and visualize the results. Before moving to the challenge, let us have a brief of what Total column water vapour is and how useful it is to atmospheric science. 

#### Total column water vapour (TCWV)

Total column water vapour (TCWV) refers to the amount of water vapour present in a vertical column of the atmosphere that extends from the Earth's surface to the top of the atmosphere. It is usually expressed in units of millimetres or centimetres of water vapour thickness over a unit area.

TCWV is a critical parameter in atmospheric science, as it plays a crucial role in the Earth's energy balance and the water cycle. It affects the formation of clouds and precipitation, and it is an essential component of climate models. Some of the ways in which TCWV is useful to include:

- **Climate research**: TCWV is a crucial component of climate models, which are used to understand and predict long-term changes in the Earth's climate. Accurate measurements of TCWV help to improve the accuracy of these models and provide insights into the role of water vapour in the Earth's climate system.

- **Weather forecasting**: TCWV measurements can be used to predict short-term weather events, such as heavy rainfall, thunderstorms, and tropical cyclones. This information is particularly important for forecasting severe weather events, which can cause significant damage to property and human life.

- **Agriculture**: TCWV measurements can be used to monitor soil moisture levels, which are critical for crop growth and yield. Accurate measurements of TCWV can help farmers to optimize irrigation schedules and reduce water usage, which can lead to increased crop productivity and profitability.

- **Aviation**: TCWV measurements are used in aviation to improve safety and efficiency. Water vapour in the atmosphere can cause turbulence and affect the performance of aircraft engines. Accurate TCWV measurements can help pilots to avoid areas of turbulence and adjust their flight paths to minimize fuel consumption.

Overall, TCWV is a crucial parameter for understanding the Earth's atmospheric system, and it provides essential information which is used for making informed decisions in a range of industries.

#### 1. Converting outputs for TCWV and storing them in the assets folder

In [ ]:
# Converting the outputs from h5py file to images and storing them in the assets folder
import h5py
import numpy as np
import multiprocessing
from tqdm import tqdm
import matplotlib.pyplot as plt

variable=['u10','v10','t2m','sp','msl','t_850','u_1000','v_1000','z_1000','u_850','v_850',
            'z_850','u_500','v_500','z_500','t_500','z_50','r_500','r_850','tcwv']

filename = "../../source_code/FCN/output/autoregressive_predictions_z500_vis.h5"
hf = h5py.File(filename, 'r')
data = hf.get('predicted')[0]

####### TODO ########
j=0 
###################### 
    
for i in tqdm(range(data.shape[0])):
    # Get the graph 
    plt.imshow(data[i][j],cmap='gray')
    plt.axis('off')
    # Save as image
    store_loc = "/../../source_code/extension/assets/FCN_output/"+variable[j]+'/'
    if not os.path.exists(os.getcwd()+store_loc):
        os.makedirs(os.getcwd()+store_loc)
    plt.savefig(os.getcwd()+store_loc+str(i)+'.jpg',bbox_inches='tight', pad_inches=0)
plt.close()
hf.close()

```python
##################### Solution #####################
# Converting the outputs from h5py file to images and storing them in the assets folder
import h5py
import numpy as np
import multiprocessing
from tqdm import tqdm
import matplotlib.pyplot as plt

variable=['u10','v10','t2m','sp','msl','t_850','u_1000','v_1000','z_1000','u_850','v_850',
            'z_850','u_500','v_500','z_500','t_500','z_50','r_500','r_850','tcwv']

filename = "../../source_code/FCN/output/autoregressive_predictions_z500_vis.h5"
hf = h5py.File(filename, 'r')
data = hf.get('predicted')[0]

####### TODO ########
j=19
###################### 
    
for i in tqdm(range(data.shape[0])):
    # Get the graph 
    plt.imshow(data[i][j],cmap='gray')
    plt.axis('off')
    # Save as image
    store_loc = "/../../source_code/extension/assets/FCN_output/"+variable[j]+'/'
    if not os.path.exists(os.getcwd()+store_loc):
        os.makedirs(os.getcwd()+store_loc)
    plt.savefig(os.getcwd()+store_loc+str(i)+'.jpg',bbox_inches='tight', pad_inches=0)
plt.close()
hf.close()
```

#### 2. Modifying the extension to use the inputs for Total column water vapor


Modify the extension here - [`extension.py`](../../source_code/extension/omniverse-project-navier-stokes/exts/omniverse.extension.navierstokes/omniverse/extension/navierstokes/extension.py)

Since it autoloads everytime you save, you can simple save and head over to Omniverse window to visualise the results.

```python
import omni.ext
import omni.ui as ui
from pxr import Usd, UsdShade, Sdf
import numpy as np
import os 

# Any class derived from `omni.ext.IExt` in top level module (defined in `python.modules` of `extension.toml`) will be
# instantiated when extension gets enabled and `on_startup(ext_id)` will be called. Later when extension gets disabled
# on_shutdown() is called.
class OmniverseExtensionNavierstokesExtension(omni.ext.IExt):
    # ext_id is current extension id. It can be used with extension manager to query additional information, like where
    # this extension is located on filesystem.
    def on_startup(self, ext_id):
        print("[omniverse.extension.navierstokes] omniverse extension navierstokes startup")
        
        # Get shader input for opacity map
        stage = omni.usd.get_context().get_stage()
        shader = UsdShade.Shader.Get(stage,"/World/Looks/Clouds/Shader")
        texture = shader.GetInput("opacity_texture")
        
        # Get and store image paths 
        files=[]
        for i in range(30):
            ##################### TODO #####################
            files.append(os.getcwd()+'/../extension/assets/FCN_output/u10/'+str(i)+'.jpg')
            ################################################
        
        # Assign images every 5 timesteps
        texture.GetAttr().Clear()
        for i in range(30):
            texture.Set(Sdf.AssetPath(files[i]),Usd.TimeCode(5*i))
                        
        # Trim Timeline
        stage.SetStartTimeCode(0)
        stage.SetEndTimeCode(150)
        
        
    def on_shutdown(self):
        print("[omniverse.extension.navierstokes] omniverse extension navierstokes shutdown")


```

```python
##################### Solution #####################
import omni.ext
import omni.ui as ui
from pxr import Usd, UsdShade, Sdf
import numpy as np
import os 

# Any class derived from `omni.ext.IExt` in top level module (defined in `python.modules` of `extension.toml`) will be
# instantiated when extension gets enabled and `on_startup(ext_id)` will be called. Later when extension gets disabled
# on_shutdown() is called.
class OmniverseExtensionNavierstokesExtension(omni.ext.IExt):
    # ext_id is current extension id. It can be used with extension manager to query additional information, like where
    # this extension is located on filesystem.
    def on_startup(self, ext_id):
        print("[omniverse.extension.navierstokes] omniverse extension navierstokes startup")
        
        # Get shader input for opacity map
        stage = omni.usd.get_context().get_stage()
        shader = UsdShade.Shader.Get(stage,"/World/Looks/Clouds/Shader")
        texture = shader.GetInput("opacity_texture")
        
        # Get and store image paths 
        files=[]
        for i in range(32):
            ##################### TODO #####################
            files.append(os.getcwd()+'/../extension/assets/FCN_output/tcwv/'+str(i)+'.jpg')
            ################################################
        
        # Assign images every 5 timesteps
        texture.GetAttr().Clear()
        for i in range(32):
            texture.Set(Sdf.AssetPath(files[i]),Usd.TimeCode(5*i))
                        
        # Trim Timeline
        stage.SetStartTimeCode(0)
        stage.SetEndTimeCode(160)
        
        
    def on_shutdown(self):
        print("[omniverse.extension.navierstokes] omniverse extension navierstokes shutdown")


```

### Optional

#### 1. Setting a blue tint

Since we are visualising the Total column water vapor, we can set the opacity map to have a blue tint as a representation to the water vapor, this can be done by selecting our `Clouds` Material and scrolling down to `Albedo` section and setting the color tint to a slightly blue shade. 

<center><img src="images/ext11.png" alt="Drawing" style="width:350px" /></center>

#### 2. Using the Movie Capture Extension

This extension allows us to render visually beautiful outputs as a video file, let us now enable this extension and create a render. 

Select `Window->Extensions` to open the extensions window and search for `Movie Capture` and enable the extension.

<center><img src="images/ext12.png" alt="Drawing" style="width:800px" /></center>


Once the Movie Capture extension is enabled, you can notice the Movie Capture window is displayed, you can set the following parameters. 
- **FPS**: You can choose an FPS, a `30 FPS` setting will create a 5 second clip as we have used 150 frames.
- **Render Preset**: We can set it to `RTX-Interactive` for a much visually appealing render. 
- **Path**: We can then set the path to where we want our output render to be saved, since all our files are in the `/workspace/python/source_code/extension/` folder, we can set it to save the file in this folder. 
- **Name**: We can set it to `tcwv` and give the `.mp4` extension. 

Once we change all the values, we can go ahead and `Capture Sequence`, it will take a few minutes to render and store the video file. 

<center><img src="images/ext13.png" alt="Drawing" style="width:350px" /></center>


The final render would look close to the below video clip.

In [1]:
from IPython.display import Video
Video("images/tcwv.mp4",width=1200, height=600,html_attributes='controls loop autoplay')

Congratulations on successfully developing the extensions with NVIDIA Omniverse! 

While the scope of the bootcamp is to introduce you to Omniverse and build an extension to visualise the results, we suggest you get creative and try implementing additional modifications, such as integrating a skybox and incorporating a dropdown menu to enable the selection of variables for display. 

Kindly go back to the first notebook to interrupt the Omniverse command once you have completed the bootcamp.

--- 

Don't forget to check out additional [Open Hackathons Resources](https://www.openhackathons.org/s/technical-resources) and join our [OpenACC and Hackathons Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

---

# Licensing

Copyright © 2023 OpenACC-Standard.org.  This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0). These materials may include references to hardware and software developed by other entities; all applicable licensing and copyrights apply.
